<a href="https://colab.research.google.com/github/kike22-22/IA/blob/main/Examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Paso 1: Instalación de dependencias
!pip install tensorflowjs
!pip install tensorflow==2.16.0rc0

# Paso 2: Importación de librerías y configuración del entorno
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
# Import EarlyStopping from tensorflow.keras.callbacks
from tensorflow.keras.callbacks import EarlyStopping
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# Configuración para ocultar mensajes de advertencia
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Paso 3: Crear carpeta para los archivos generados
output_dir = "historial_credicticio"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ... (rest of your code remains the same)

# Paso 4: Definir datos de entrada y salida, todos estos datos, son para mejorar la prescision de la prediccion
entradas = np.array([
    [3000, 1000, 12], [4000, 500, 24], [2500, 1500, 6], [5000, 0, 36],
    [3500, 200, 48], [6000, 700, 12], [4500, 1300, 24], [10000, 100, 1],
    [5000, 300, 12], [1000, 5000, 6], [7000, 500, 12], [2000, 3000, 24],
    [3000, 250, 12], [4500, 600, 6], [5500, 1200, 48], [1500, 500, 6],
    [2800, 700, 24], [4800, 300, 36], [7200, 200, 12], [6800, 1500, 18],
    [5900, 400, 24], [2400, 800, 36], [8000, 600, 12], [3300, 200, 6],
    [4700, 1500, 24], [1000, 3500, 6], [4000, 900, 18], [9000, 120, 24]
], dtype=float)

# Salidas (1: Buen historial, 0: Mal historial)
historico_crediticio = np.array([
    1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
    0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
    1, 0, 1, 1
], dtype=float)

# Normalización de datos
scaler = StandardScaler()
entradas_escaladas = scaler.fit_transform(entradas)

# División de los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(entradas_escaladas, historico_crediticio, test_size=0.25, random_state=42)

# Paso 5: Construcción del modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(3,), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilación del modelo
modelo.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Configuración del entrenamiento con EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenamiento del modelo
print("Comenzando el entrenamiento...")
historial = modelo.fit(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), verbose=False, callbacks=[early_stopping])
print("Entrenamiento completado.")

# Evaluación del modelo
loss, accuracy = modelo.evaluate(X_val, y_val)
print(f'Precisión en el conjunto de validación: {accuracy * 100:.2f}%')

# Guardado del modelo para TensorFlow.js
modelo.save(f"{output_dir}/modelo_crediticio_v2.h5")
!tensorflowjs_converter --input_format=keras {output_dir}/modelo_crediticio_v2.h5 {output_dir}

# Prueba con nuevas entradas
entradas_de_prueba = np.array([
    [8500, 200, 12], [3500, 1000, 18], [5000, 1500, 24], [4500, 700, 36]
])

entradas_de_prueba_escaladas = scaler.transform(entradas_de_prueba)
predicciones = modelo.predict(entradas_de_prueba_escaladas)

# Imprimir resultados
for entrada, prediccion in zip(entradas_de_prueba, predicciones):
    estado = "Bueno" if prediccion[0] >= 0.5 else "Malo"
    print(f"Entrada: {entrada} -> Predicción: Historial Crediticio {estado}")


  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.0rc0
    Uninstalling tensorflow-2.16.0rc0:
      Successfully uninstalled tensorflow-2.16.0rc0
  Using cached tensorflow-2.16.0rc0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.16.0rc0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
Using cached tensorbo

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Comenzando el entrenamiento...
Entrenamiento completado.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5714 - loss: 0.6395


Precisión en el conjunto de validación: 57.14%
failed to lookup keras version from the file,
    this is likely a weight only file
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Entrada: [8500  200   12] -> Predicción: Historial Crediticio Bueno
Entrada: [3500 1000   18] -> Predicción: Historial Crediticio Bueno
Entrada: [5000 1500   24] -> Predicción: Historial Crediticio Bueno
Entrada: [4500  700   36] -> Predicción: Historial Crediticio Bueno
